In [1]:
# What do I want to see with tooling? What is the argument I want to make?

# The model outputs normalized returns for each area for each eval year
# I want to take out one target year, for e.g. 2022
# I want to sort the relative returns for that year
# I want to compare several areas
# I want to see the cumulative factors for each area, this will help me identify trends

# once this is done, write a narrative for the teammates

# target_year = 2022
# prefecture_code = 13 # (Tokyo, 13), (Hokaido, 1)
# results_df = output_df[output_df["year"] == target_year].sort_values(by=["predicted_normalized_return"], ascending=False)
# prefectural_results_df = results_df[(results_df["area_code"].astype(int) // 1000 == prefecture_code)]
# # results_df
# print(len(prefectural_results_df))
# prefectural_results_df.head(10)
# prefectural_results_df.tail(10)

In [150]:
import warnings

import pandas as pd
import numpy as np

from jre_utils.datapath import (
    factor_data_paths,
    model_output_data_paths,
    get_price_index_path
)

from jre_utils.visualize import plot_time_series
from jre_utils.process import get_cumulative_growth, get_cumulative_growth_from_base


warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [141]:
investment_asset_type = "building"
dataset_key = "transactions"
years_ahead = 4
test_year = 2022

metrics = {
    "median": "unit_price_median",
    "gmean": "unit_price_gmean",
    "robust": "robust_price_index",
    "ols": "ols_price_index",
}

# granularity_columns = ["area", "area_code"]
granularity_columns = ["area_code"]
group_by_columns = granularity_columns + ["year"]

metric_key = "robust"
metric = metrics[metric_key]
metric_pct_chg = metric + "_pct_chg"

get_data_path = get_price_index_path


In [142]:
population_path = factor_data_paths["processed"]["population"]["municipality"]
migration_path = factor_data_paths["processed"]["migration"]["municipality"]
taxable_income_path = factor_data_paths["processed"]["taxable_income"]["municipality"]
new_dwellings_path = factor_data_paths["processed"]["new_dwellings"]["municipality"]
lfs_revenue_path = factor_data_paths["processed"]["lfs_revenue_breakdown"][
    "municipality"
]



In [143]:
df =  pd.read_csv(get_data_path(investment_asset_type))
df = df.sort_values(by=group_by_columns, ascending=True)
df = df[group_by_columns + [metric, "count", "robust_weights"]]
df["asset_type"] = investment_asset_type

population_df = pd.read_csv(population_path)
migration_df = pd.read_csv(migration_path)
taxable_income_df = pd.read_csv(taxable_income_path)
new_dwellings_df = pd.read_csv(new_dwellings_path)
lfs_revenue_df= pd.read_csv(lfs_revenue_path)

df = (
    df.merge(population_df, on=group_by_columns, how="left")
    .merge(migration_df, on=group_by_columns, how="left")
    .merge(taxable_income_df, on=group_by_columns, how="left")
    .merge(new_dwellings_df, on=group_by_columns, how="left")
    .merge(lfs_revenue_df, on=group_by_columns, how="left")
)

In [144]:
dataset_name = (
    f"sequence_{dataset_key}_{investment_asset_type}_{metric_key}_{years_ahead}"
)
output_dataset_name = f"{dataset_name}_{test_year}"
model_output_data_path = model_output_data_paths[output_dataset_name]
output_df = pd.read_csv(model_output_data_path)
output_df = output_df.sort_values(by=["predicted_normalized_return"], ascending=False)


In [145]:
from jre_utils.process import get_most_active_municipalities


pop_500_df = get_most_active_municipalities(output_df.merge(population_df, on=["year", "area_code"]), "population", 500)
output_df_top_100 = pop_500_df.head(100)
output_df_bottom_100 = pop_500_df.tail(100)

# output_df_top_100 = output_df.head(100)
# output_df_bottom_100 = output_df.tail(100)

top_areas = output_df_top_100.sample(10)["area_code"].values
bottom_areas = output_df_bottom_100.sample(10)["area_code"].values


In [146]:
top_areas

array([11224, 20202, 13201, 11229, 40130, 28201, 27205, 27140, 13208,
       25206])

In [147]:
bottom_areas

array([ 1203,  7207,  1205,  6203,  7213, 43215, 13205, 12202, 15205,
        4212])

In [151]:
# Top vs Bottom
highlight_color = "#caf0f8"
color_map = {f"{area_code}": "#023e8a" for area_code in top_areas}
color_map.update({f"{area_code}": "#48cae4" for area_code in bottom_areas})
area_codes = top_areas.tolist() + bottom_areas.tolist()

# Arbitrary

# color_map = {
#     "('Tokyo-to Minato-ku', 13103)": "#03045e",
#     "('Hokkaido Sapporo-shi', 1100)": "#0077b6",
#     "('Chiba-ken Isumi-shi', 12238)": "#00b4d8",
# }

# area_codes = [13103, 1100, 12238]

area_df = df[df["area_code"].isin(area_codes)].sort_values(by="year", ascending=True)
area_df[metric] = area_df[metric] / 100

plot_time_series(
    area_df,
    f"{metric}",
    group_by_columns,
    granularity_columns,
    f"Price Index over time",
    width=1000,
    height=600,
    highlight=True,
    highlight_range=(2019, 2022),
    highlight_color=highlight_color,
    color_discrete_map=color_map,
)

# Set 13103 to red, 1100 to blue, 12238 to green

In [111]:

plot_time_series(
    area_df,
    f"{metric}",
    group_by_columns,
    granularity_columns,
    f"Price Index over time",
    width=1000,
    height=600,
    # highlight=True,
    # highlight_range=(2019, 2022),
    # highlight_color=highlight_color,
    color_discrete_map=color_map
    
)

growth_factors = [
    # "taxable_income_growth",
    # "total_tax_growth",
    # "net_migration_ratio",
    # "new_dwellings_ratio",
]

base_factors = [
    # "total_tax",
    "taxpayer_count",
    # "count",
    "population",
    "taxable_income",
    "existing_dwellings",
]

cumulative_factors = [
    f"cumulative_{factor}" for factor in growth_factors
] + [
    f"cumulative_{factor}_growth" for factor in base_factors
]

area_df[growth_factors] = area_df[growth_factors].replace({0: np.nan})
area_df[base_factors] = area_df[base_factors].replace({0: np.nan})

for factor in growth_factors:
    area_df[f"cumulative_{factor}"] = get_cumulative_growth(
        area_df.copy(), factor
    )

for factor in base_factors:
    area_df[f"cumulative_{factor}_growth"] = get_cumulative_growth_from_base(
        area_df.copy(), factor
    )

for cumulative_factor in cumulative_factors:
    plot_time_series(
        area_df[area_df["year"] <= 2021],
        cumulative_factor,
        group_by_columns,
        granularity_columns,
        f"{cumulative_factor} over time",
        # visible="legendonly",
        width=1000,
        height=600,
        # highlight=True,
        # highlight_range=(2013, 2018),
        # highlight_color=highlight_color,
        color_discrete_map=color_map
    )

# color_discrete_map={
#             "('Tokyo-to Minato-ku', 13103)": "#EF553B",
#             "('Hokkaido Sapporo-shi', 1100)": "#636efa",
#             "('Chiba-ken Isumi-shi', 12238)": "#00cc96",
#         }
    



In [11]:
# use population as the error scaling parameter instead

# autocorrelation over time series
# cross time series lead lag for pairs

# Forward looking smoothing for prediction column
# backward looking smoothing for factor column

# time series of all transa


In [11]:
# area_df